In [41]:
from datetime import date

import yfinance as yf
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from plotly import graph_objs as go

START = "2007-02-24"
TODAY = "2022-02-24"



stocks = ("GC=F")



period = 3 * 365


def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data


data = load_data(stocks)



[*********************100%***********************]  1 of 1 completed


In [42]:
def plot_raw_data():
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name='stock_close'))
    fig.layout.update(title_text='Time Series Data', xaxis_rangeslider_visible=True)
    fig.show()
plot_raw_data()

In [39]:
#Forecasting
df_train = data[['Date', 'Close']]
df_train = df_train.rename(columns={'Date': 'ds', 'Close':'y'})

changepoint_prior_scale = 2
seasonality_prior_scale = 10

m = Prophet(seasonality_prior_scale=seasonality_prior_scale,changepoint_prior_scale=changepoint_prior_scale)

m.fit(df_train)
future = m.make_future_dataframe(periods=period)
forecast = m.predict(future)




fig1 = plot_plotly(m, forecast)
fig1.show()



fig2 = m.plot_components(forecast)

fig2.show()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<ipython-input-39-50bbe4784a1e>:24: UserWarning:

Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.



In [40]:
import numpy as np
se = np.square(forecast.loc[:, 'yhat'] - data['Close'])
mse = np.mean(se)
rmse = (np.sqrt(mse)).round(5)
print('USD Index RMSE:' + f' {rmse}')

USD Index RMSE: 0.03016


### Us Dollar Index
#### RMSE: 1.608 Cents 

FB Prophet Tuning

- changepoint_prior_scale = 2
- seasonality_prior_scale = 20            

### Euro/US Dollar Price
#### RMSE: 3.016 Cents

FP Prophet Tuning
- changepoint_prior_scale = 2
- seasonality_prior_scale = 10

### Gold Price
#### RMSE: 63.31 Dollars 

FB Prophet Tuning

- changepoint_prior_scale = 2
- seasonality_prior_scale = 5

### Dow Jones Index
#### RMSE: 1043.998

FB Prophet Tuning

- changepoint_prior_scale = 4
- seasonality_prior_scale = 4